In [ ]:
import pandas as pd
import torch
import torch.nn as nn
df=pd.read_csv('chessData.csv')

In [ ]:
df=df[10000:20000]
df

In [ ]:
eval=eval.to_list()

In [ ]:
eval=df['Evaluation']
eval=eval.to_list()

In [ ]:
from utils.decode_FEN import FEN_to_arr
positions=[]
extras=[]
for i in df['FEN']:
    pos,extr=FEN_to_arr(i)
    positions.append(pos)
    extras.append(extr)
    

In [ ]:
for i in range (len(eval)):
    if not eval[i][1:].isdigit(): eval[i]='+1000'
    eval[i]=int(eval[i])
eval

In [ ]:
import numpy as np
eval=eval.astype('int')
eval=np.array(eval)

# Trying with PyTorch

In [ ]:
eval=torch.tensor(eval).float()
extras=torch.tensor(extras).float()
positions=torch.tensor(positions).float()

In [ ]:
class ChessMovePredictor(nn.Module):
    def __init__(self):
        super(ChessMovePredictor, self).__init__()
        
        
        # Layer 2: Conv2D
        self.conv1 = nn.Conv2d(in_channels=6, out_channels=64, kernel_size=(8, 8), padding="same")
        self.bn1 = nn.BatchNorm2d(64, momentum=0.99, eps=1e-05)
        
        # Layer 3: Conv2D
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(8, 8), padding="same")
        self.bn2 = nn.BatchNorm2d(64, momentum=0.99, eps=1e-05)
        
        # Layer 4: Flatten
        self.flatten = nn.Flatten()
        
        # Layer 5: Input Layer for the second input
        # self.input2_layer = nn.Linear(6)
        
        # # Layer 6: Concatenate
        # self.concat = nn.cat()
        
        # Layer 7-11: Dense Layers
        self.dense1 = nn.Linear(4101, 1024)
        self.dense2 = nn.Linear(1024, 512)
        self.dense3 = nn.Linear(512, 256)
        self.dense4 = nn.Linear(256, 256)
        self.output_layer = nn.Linear(256, 1)
        self.relu = nn.ReLU()

        
    def forward(self, x1, x2):
        # Forward pass through Layer 1-4 for the first input
        x1 = self.conv1(x1)
        x1 = self.bn1(x1)
        x1 = self.conv2(x1)
        x1 = self.bn2(x1)
        x1 = self.flatten(x1)
        
        # # Forward pass through Layer 5 for the second input
        # x2 = self.input2_layer(x2)
        
        # Forward pass through Layer 6 (Concatenate)
        x = torch.cat((x1, x2),1)
        
        # Forward pass through Layer 7-11 (Dense Layers)
        x = self.relu(self.dense1(x))
        x = self.relu(self.dense2(x))
        x = self.relu(self.dense3(x))
        x = self.relu(self.dense4(x))
        output = self.output_layer(x)
        
        return output

# Instantiate the model
model = ChessMovePredictor()

optimizer = torch.optim.Adam(model.parameters(),lr=0.25)



In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of Trainable Parameters:", total_params)

In [ ]:

loss_history=[]
mse_loss = nn.MSELoss()
for i in range (100):
    eval_sample=eval[:5000]
    positions_sample=positions[:5000]
    extras_sample=extras[:5000]
    
    
    
    
    optimizer.zero_grad()
    pred=model(positions_sample,extras_sample)
    loss=mse_loss(pred,eval_sample)
    loss.backward()
    optimizer.step()
    loss_history.append(loss)
    print(loss)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
scores= pd.Series([i.item() for i in loss_history], name="scores_Actor")
scores.describe()


In [ ]:
fig, ax = plt.subplots(1, 1)
_ = scores.plot(ax=ax, label="scores_Actor")
_ = (scores.rolling(window=100)
           .mean()
           .rename("Rolling Average")
           .plot(ax=ax))
ax.legend()
_ = ax.set_xlabel("Episode Number")
_ = ax.set_ylabel("scores_Actor")

# Trying with tensorflow

In [ ]:
eval=np.stack(eval)
extras=np.stack(extras)
positions=np.stack(positions)

In [36]:
import tensorflow as tf
import numpy as np

input1 = tf.keras.layers.Input(shape=(6,8,8))
# shape1 = tf.keras.layers.Reshape(target_shape=(8, 8, 6))(input1)
conv1 = tf.keras.layers.Conv2D(kernel_size=(8,8), padding="same", activation="relu", filters=64, input_shape=(8,8,1))(input1)
bn1 = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-05)(conv1)
conv2 = tf.keras.layers.Conv2D(kernel_size=(8,8), padding="same", activation="relu", filters=64, input_shape=(8,8,1))(bn1)
bn2 = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-05)(conv2)
flatten1 = tf.keras.layers.Flatten()(bn2)
input2 = tf.keras.layers.Input(shape=(5,))

conc = tf.keras.layers.concatenate([flatten1,input2])

Denselayer1 = tf.keras.layers.Dense(1024, activation='relu')(conc)
# Denselayer2 = tf.keras.layers.Dense(512, activation='relu')(Denselayer1)
# Denselayer3 = tf.keras.layers.Dense(256, activation='relu')(Denselayer2)
# Denselayer4 = tf.keras.layers.Dense(256, activation='relu')(Denselayer3)
Output = tf.keras.layers.Dense(1, activation='linear')(Denselayer1)



data_model = tf.keras.models.Model(inputs=[input1, input2], outputs=Output)

# predictions = data_model([(inputboard[:1]), (inputmeta[:1])]).numpy

metric =[tf.keras.metrics.MeanAbsoluteError()]





#Removing the Clipnorm of clipnorm=1 may make training faster
#opt = tf.keras.optimizers.Adam(clipnorm=1)

opt = tf.keras.optimizers.Adam()




los = tf.keras.losses.MeanSquaredError()

data_model.compile(optimizer=opt, 
                   loss=los,
                   metrics=metric)
data_model.summary()
# data_model.fit([inputboard, inputmeta], data_labels, epochs=1000, batch_size=8192, shuffle=True)

# data_model.save("engine01")

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 6, 8, 8)]    0           []                               
                                                                                                  
 conv2d_8 (Conv2D)              (None, 6, 8, 64)     32832       ['input_9[0][0]']                
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 6, 8, 64)    256         ['conv2d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 conv2d_9 (Conv2D)              (None, 6, 8, 64)     262208      ['batch_normalization_8[0][

In [37]:
data_model.fit([positions[:9500], extras[:9500]], eval[:9500], epochs=100, validation_data=([positions[9500:], extras[9500:]], eval[9500:]) ,batch_size=8192, shuffle=True)

# data_model.save("engine01")

Epoch 1/100
2/2 [==============================] - 22s 4s/step - loss: 370555.2812 - mean_absolute_error: 284.5161 - val_loss: 143654.9375 - val_mean_absolute_error: 218.9357
Epoch 2/100
2/2 [==============================] - 23s 4s/step - loss: 358714.2188 - mean_absolute_error: 277.3796 - val_loss: 143387.9688 - val_mean_absolute_error: 218.6862
Epoch 3/100
2/2 [==============================] - 26s 3s/step - loss: 350155.2812 - mean_absolute_error: 297.1836 - val_loss: 143261.3438 - val_mean_absolute_error: 218.7084
Epoch 4/100
2/2 [==============================] - 25s 4s/step - loss: 342262.1875 - mean_absolute_error: 302.3872 - val_loss: 143310.3906 - val_mean_absolute_error: 218.8879
Epoch 5/100
2/2 [==============================] - 25s 3s/step - loss: 329251.9375 - mean_absolute_error: 276.5130 - val_loss: 143185.9219 - val_mean_absolute_error: 218.8744
Epoch 6/100
2/2 [==============================] - 24s 4s/step - loss: 316711.3125 - mean_absolute_error: 260.1830 - val_loss

KeyboardInterrupt: 

In [30]:
pred=data_model.predict([positions[100:200],extras[100:200]])

4/4 [==============================] - 1s 22ms/step


In [32]:
tf.keras.losses.mae(eval[100:200],pred)

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([150.2159 , 151.89499, 150.34349, 150.99942, 150.26477, 151.86908,
       150.21   , 151.76602, 150.87686, 150.84691, 151.0691 , 150.75969,
       150.67967, 151.22668, 150.39635, 150.66708, 152.01247, 150.26784,
       152.16908, 150.5117 , 151.11232, 151.14258, 150.21219, 151.04903,
       150.86034, 151.41896, 150.81696, 150.25232, 151.34113, 151.40695,
       150.73305, 150.8349 , 150.70406, 150.40224, 151.29892, 150.31676,
       151.3078 , 150.2555 , 151.32483, 150.96211, 152.69647, 150.23701,
       151.84142, 150.56471, 150.89053, 150.43225, 151.31436, 150.41504,
       151.61224, 150.24232, 151.46129, 150.22014, 152.73276, 150.28444,
       152.67119, 150.54617, 150.89508, 150.37766, 151.01553, 150.66339,
       150.87843, 150.2369 , 151.521  , 150.30655, 151.17404, 150.8422 ,
       150.63673, 150.61893, 151.35849, 150.4205 , 155.23238, 150.998  ,
       152.11047, 150.2251 , 151.89786, 152.86531, 151.59998, 150.21   ,
   